In [1]:
import pandas as pd
import re
import nltk
import warnings
warnings.filterwarnings('ignore')

# Download stopwords for Russian language / Скачиваем стоп-слова для русского языка
nltk.download('stopwords')
from nltk.corpus import stopwords

print("MASTER'S PROJECT: Toxic messages handling project")
print("="*70)

# =============================================================================
# 1. DATA LOADING - EXCEL FILE / ЗАГРУЗКА ДАННЫХ ИЗ EXCEL
# =============================================================================

print("📥 Loading data from Excel file...")
file_path = "MergedDS.xlsx"  

try:
    # Try to read Excel file / Пробуем прочитать Excel файл
    df = pd.read_excel(file_path)
    print("✅ Successfully loaded from Excel")
    
except Exception as e:
    print(f"❌ Excel loading failed: {e}")
    print("🔄 Trying CSV with different separators...")
    
    # Fallback to CSV with different separators / Резервный вариант с разными разделителями
    separators = ['\t', ',', ';', '|']
    
    for sep in separators:
        try:
            df = pd.read_csv(file_path.replace('.xlsx', '.csv'), sep=sep, encoding='utf-8')
            print(f"✅ Loaded with separator: '{sep}'")
            break
        except:
            continue
    else:
        print("❌ All loading methods failed")
        exit()

print(f"Original dataset size: {df.shape}")

# Basic information / Базовая информация
print("\n📊 Basic dataset information:")
print(f"Columns: {df.columns.tolist()}")
print(f"Data types:\n{df.dtypes}")

# Display first few rows to check data / Показываем первые строки для проверки данных
print("\n👀 First 5 rows of data:")
print(df.head())

# Check if we have the expected columns / Проверяем наличие ожидаемых колонок
expected_columns = ['text_raw', 'is_toxic', 'toxicity_type', 'source_platform']
available_columns = [col for col in expected_columns if col in df.columns]
print(f"\n✅ Available expected columns: {available_columns}")

# Show sample data from key columns / Показываем примеры данных из ключевых колонок
if 'text_raw' in df.columns:
    print(f"\n📝 Sample text_raw values:")
    for i in range(min(3, len(df))):
        text_sample = str(df['text_raw'].iloc[i])[:100]
        print(f"  {i+1}: {text_sample}...")

if 'is_toxic' in df.columns:
    print(f"\n🔢 is_toxic unique values: {df['is_toxic'].unique()}")

if 'toxicity_type' in df.columns:
    print(f"🏷️ toxicity_type unique values: {df['toxicity_type'].unique()}")

if 'source_platform' in df.columns:
    print(f"🌐 source_platform unique values: {df['source_platform'].unique()[:5]}")

# Missing values analysis / Анализ пропусков
print(f"\n🔍 Missing values:")
missing_data = df.isnull().sum()
for col, count in missing_data.items():
    if count > 0:
        print(f"  {col}: {count} missing ({count/len(df)*100:.1f}%)")


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\anton\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


MASTER'S PROJECT: Toxic messages handling project
📥 Loading data from Excel file...
✅ Successfully loaded from Excel
Original dataset size: (469967, 9)

📊 Basic dataset information:
Columns: ['Column1', 'raw_text_id', 'dataset_id', 'source_platform', 'nickname', 'is_verified', 'text_raw', 'is_toxic', 'toxicity_type']
Data types:
Column1              int64
raw_text_id          int64
dataset_id           int64
source_platform     object
nickname            object
is_verified          int64
text_raw            object
is_toxic           float64
toxicity_type       object
dtype: object

👀 First 5 rows of data:
   Column1  raw_text_id  dataset_id source_platform nickname  is_verified  \
0        0            0           0     2ch, pikabu      NaN            1   
1        1            1           0     2ch, pikabu      NaN            1   
2        2            2           0     2ch, pikabu      NaN            1   
3        3            3           0     2ch, pikabu      NaN            1   
4 

In [3]:

# =============================================================================
# 2. DATA CLEANING AND TYPE CONVERSION / ОЧИСТКА ДАННЫХ И КОНВЕРТАЦИЯ ТИПОВ
# =============================================================================

print("\n🔧 Data cleaning and type conversion...")

# Clean column names / Очистка названий колонок
df.columns = df.columns.str.strip()
print(f"Cleaned column names: {df.columns.tolist()}")

# Convert is_toxic to numeric / Конвертируем is_toxic в числовой формат
if 'is_toxic' in df.columns:
    print(f"is_toxic before conversion: {df['is_toxic'].unique()}")
    df['is_toxic'] = pd.to_numeric(df['is_toxic'], errors='coerce')
    print(f"is_toxic after conversion: {df['is_toxic'].unique()}")

# Clean text_raw column / Очистка колонки text_raw
if 'text_raw' in df.columns:
    # Remove extra quotes and whitespace / Удаляем лишние кавычки и пробелы
    df['text_raw'] = df['text_raw'].astype(str).str.strip().str.strip('"')
    print("Cleaned text_raw column")

# Convert other numeric columns / Конвертируем другие числовые колонки
numeric_columns = ['raw_text_id', 'dataset_id', 'is_verified']
for col in numeric_columns:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')



🔧 Data cleaning and type conversion...
Cleaned column names: ['Column1', 'raw_text_id', 'dataset_id', 'source_platform', 'nickname', 'is_verified', 'text_raw', 'is_toxic', 'toxicity_type']
is_toxic before conversion: [ 1.  0. nan]
is_toxic after conversion: [ 1.  0. nan]
Cleaned text_raw column


In [5]:

# =============================================================================
# 3. MISSING LABELS HANDLING / ОБРАБОТКА ПРОПУЩЕННЫХ МЕТОК
# =============================================================================

print("\n🏷️ Processing missing labels...")

if 'is_toxic' in df.columns and 'toxicity_type' in df.columns:
    # Check current state of labels / Проверяем текущее состояние меток
    print(f"is_toxic unique values: {df['is_toxic'].unique()}")
    print(f"toxicity_type unique values: {df['toxicity_type'].unique()}")

    # Logic for filling missing values: / Логика заполнения пропусков:
    # 1. If is_toxic is empty but toxicity_type is filled (not empty and not NORMAL) → is_toxic = 1
    # 1. Если is_toxic пустое, но toxicity_type заполнено (не пустое и не NORMAL) → is_toxic = 1
    mask_toxic_missing = df['is_toxic'].isna()
    mask_tox_type_filled = (~df['toxicity_type'].isna()) & (df['toxicity_type'] != '') & (df['toxicity_type'] != 'NORMAL')
    
    fill_toxic_count = (mask_toxic_missing & mask_tox_type_filled).sum()
    df.loc[mask_toxic_missing & mask_tox_type_filled, 'is_toxic'] = 1
    print(f"Filled is_toxic=1 for {fill_toxic_count} rows (toxicity_type indicates toxicity)")

    # 2. If is_toxic = 0, and toxicity_type is empty → toxicity_type = 'NORMAL'
    # 2. Если is_toxic = 0, а toxicity_type пустое → toxicity_type = 'NORMAL'
    mask_toxic_zero = df['is_toxic'] == 0
    mask_tox_type_empty = (df['toxicity_type'].isna()) | (df['toxicity_type'] == '')
    
    fill_normal_count = (mask_toxic_zero & mask_tox_type_empty).sum()
    df.loc[mask_toxic_zero & mask_tox_type_empty, 'toxicity_type'] = 'NORMAL'
    print(f"Filled toxicity_type='NORMAL' for {fill_normal_count} rows")

    # Convert is_toxic to integer / Конвертируем is_toxic в integer 
    if df['is_toxic'].dtype == 'float64':
        df['is_toxic'] = df['is_toxic'].astype('Int64')
        print("Converted is_toxic to integer type")

    # Check results after filling / Проверяем результаты после заполнения
    print(f"\n📊 After filling missing labels:")
    print(f"is_toxic missing: {df['is_toxic'].isna().sum()} ({df['is_toxic'].isna().sum()/len(df)*100:.1f}%)")
    print(f"toxicity_type missing: {df['toxicity_type'].isna().sum()} ({df['toxicity_type'].isna().sum()/len(df)*100:.1f}%)")



🏷️ Processing missing labels...
is_toxic unique values: [ 1.  0. nan]
toxicity_type unique values: [nan 'INSULT' 'NORMAL' 'OBSCENITY' 'THREAT' 'INAPPROPRIATE' 'SENSITIVE']
Filled is_toxic=1 for 124586 rows (toxicity_type indicates toxicity)
Filled toxicity_type='NORMAL' for 65333 rows
Converted is_toxic to integer type

📊 After filling missing labels:
is_toxic missing: 0 (0.0%)
toxicity_type missing: 31765 (6.8%)


In [7]:
# =============================================================================
# 4. TEXT PREPROCESSING / ПРЕДОБРАБОТКА ТЕКСТА
# =============================================================================

print(f"\n🧹 Starting text preprocessing...")

if 'text_raw' in df.columns:
    # Save initial number of rows / Сохраняем исходное количество строк
    initial_rows = len(df)

    # 1. Remove completely empty rows / Удаление полностью пустых строк
    df = df.dropna(subset=['text_raw'])
    df = df[df['text_raw'] != '']
    df = df[df['text_raw'] != 'nan']
    print(f"Removed empty texts: {initial_rows - len(df)}")

    # Text cleaning function / Функция очистки текста
    def clean_text_advanced(text):
        if pd.isna(text) or text == '' or text == 'nan':
            return ""
        
        text = str(text)
        
        # Convert to lowercase / Приведение к нижнему регистру
        text = text.lower()
        
        # HTML entities decoding / Декодирование HTML-сущностей
        import html
        try:
            text = html.unescape(text)
        except:
            pass
        
        # Remove HTML tags / Удаление HTML-тегов
        text = re.sub(r'<[^>]+>', '', text)
        
        # Remove URLs / Удаление URL
        text = re.sub(r'http\S+', ' URL ', text)
        
        # Remove emails / Удаление email
        text = re.sub(r'\S+@\S+', ' EMAIL ', text)
        
        # Replace numbers / Замена чисел
        text = re.sub(r'\d+', ' NUMBER ', text)
        
        # Process emoticons / Обработка эмотиконов
        text = re.sub(r'[:;=]-?[\)\(DP]', ' EMOTICON ', text)
        
        # Process emojis / Обработка эмодзи
        emoji_pattern = re.compile("["
            u"\U0001F600-\U0001F64F"  # emoticons / эмоции
            u"\U0001F300-\U0001F5FF"  # symbols & pictographs / символы и пиктограммы
            u"\U0001F680-\U0001F6FF"  # transport & map symbols / транспорт и карты
            u"\U0001F1E0-\U0001F1FF"  # flags (iOS) / флаги
            u"\U00002702-\U000027B0"  # other symbols / другие символы
            u"\U000024C2-\U0001F251" 
            "]+", flags=re.UNICODE)
        text = emoji_pattern.sub(r' EMOJI ', text)
        
        # Process repeated punctuation (2+ characters) / Обработка повторяющейся пунктуации (2+ символа)
        text = re.sub(r'[!?]{2,}', ' REPEAT_PUNCT ', text)  # !! ??? !!!
        text = re.sub(r'\.{2,}', ' REPEAT_PUNCT ', text)    # ... .... → REPEAT_PUNCT
        text = re.sub(r'[!?]\.+', ' REPEAT_PUNCT ', text)   # !.. ?... → REPEAT_PUNCT
        text = re.sub(r'[!?][!?]+', ' REPEAT_PUNCT ', text) # !? !!? ?! → REPEAT_PUNCT
        
        # Remove special characters but keep Russian letters / Удаление специальных символов с сохранением русских букв
        text = re.sub(r'[^\w\sа-яё]', ' ', text)
        
        # Remove extra spaces / Удаление лишних пробелов
        text = re.sub(r'\s+', ' ', text)
        text = text.strip()
        
        return text

    # Apply cleaning / Применяем очистку
    print("Cleaning texts...")
    df['clean_text'] = df['text_raw'].apply(clean_text_advanced)

    # Remove very short texts (less than 3 characters) / Удаление слишком коротких текстов
    initial_count = len(df)
    df = df[df['clean_text'].str.len() >= 3]
    print(f"After removing short texts: {len(df)} rows (removed {initial_count - len(df)})")

    # Remove text duplicates after cleaning / Удаление дубликатов текста после очистки
    initial_count = len(df)
    df = df.drop_duplicates(subset=['clean_text'])
    print(f"Removed duplicates: {initial_count - len(df)}")

    # Remove stopwords / Удаление стоп-слов
    def remove_stopwords(text):
        if not isinstance(text, str) or text == '':
            return ""
        
        russian_stopwords = set(stopwords.words('russian'))
        custom_stopwords = {
            'это', 'вот', 'как', 'так', 'и', 'в', 'над', 'к', 'до', 'не', 'на', 'но', 'за', 
            'то', 'с', 'ли', 'а', 'во', 'от', 'со', 'для', 'о', 'же', 'ну', 'вы', 'бы', 'что',
            'кто', 'он', 'она', 'они', 'оно', 'мы', 'вы', 'вас', 'вам', 'тебе', 'ты', 'мне',
            'меня', 'ему', 'ей', 'им', 'ними', 
            'NUMBER', 'URL', 'EMAIL', 'EMOTICON', 'EMOJI', 'REPEAT_PUNCT'
        }
        
        all_stopwords = russian_stopwords.union(custom_stopwords)
        words = text.split()
        filtered_words = [word for word in words if word not in all_stopwords and len(word) > 2]
        return ' '.join(filtered_words)

    print("Removing stopwords...")
    df['clean_text_no_stopwords'] = df['clean_text'].apply(remove_stopwords)

    print(f"Final dataset size after preprocessing: {len(df)} rows")


🧹 Starting text preprocessing...
Removed empty texts: 0
Cleaning texts...
After removing short texts: 469901 rows (removed 66)
Removed duplicates: 19691
Removing stopwords...
Final dataset size after preprocessing: 450210 rows


In [9]:
# =============================================================================
# 5. BASIC DATASET STATISTICS / БАЗОВАЯ СТАТИСТИКА ДАТАСЕТА
# =============================================================================

print("\n" + "="*60)
print("BASIC DATASET STATISTICS")
print("="*60)

print(f"Total records: {len(df):,}")

# Analyze is_toxic if available / Анализ is_toxic если доступно
if 'is_toxic' in df.columns:
    toxic_count = df['is_toxic'].sum()
    normal_count = len(df) - toxic_count
    print(f"Toxic records: {toxic_count:,} ({df['is_toxic'].mean():.1%})")
    print(f"Non-toxic records: {normal_count:,} ({(1 - df['is_toxic'].mean()):.1%})")

# Text length if available / Длина текстов если доступно
if 'clean_text' in df.columns:
    df['text_length'] = df['clean_text'].apply(lambda x: len(x.split()))
    df['text_length_no_stopwords'] = df['clean_text_no_stopwords'].apply(lambda x: len(x.split()))

    print(f"\nText length (in words):")
    print(f"Average: {df['text_length'].mean():.1f}")
    print(f"After stopwords removal: {df['text_length_no_stopwords'].mean():.1f}")
    print(f"Maximum: {df['text_length'].max()}")
    print(f"Minimum: {df['text_length'].min()}")

# Analyze source_platform if available / Анализ source_platform если доступно
if 'source_platform' in df.columns:
    print(f"\n🌐 Source platform analysis:")
    platform_stats = df['source_platform'].value_counts()
    for platform, count in platform_stats.head(10).items():
        percentage = count / len(df) * 100
        print(f"  {platform}: {count:,} records ({percentage:.1f}%)")

# Analyze is_verified if available / Анализ is_verified если доступно
if 'is_verified' in df.columns:
    print(f"\n✅ Verification analysis:")
    verified_stats = df['is_verified'].value_counts()
    for verified, count in verified_stats.items():
        percentage = count / len(df) * 100
        print(f"  {verified}: {count:,} records ({percentage:.1f}%)")



BASIC DATASET STATISTICS
Total records: 450,210
Toxic records: 195,151 (43.3%)
Non-toxic records: 255,059 (56.7%)

Text length (in words):
Average: 19.1
After stopwords removal: 11.0
Maximum: 3213
Minimum: 1

🌐 Source platform analysis:
  ok.ru: 244,771 records (54.4%)
  2ch.hk, Pikabu.ru, otveti.mail.ru: 124,435 records (27.6%)
  2ch, vk: 60,673 records (13.5%)
  2ch, pikabu: 14,152 records (3.1%)
  YouTube: 1,937 records (0.4%)
  Social Media, TV-Scripts (South Park): 1,450 records (0.3%)
  лентач: 981 records (0.2%)
  медуза: 932 records (0.2%)
  дождь: 879 records (0.2%)

✅ Verification analysis:
  1: 383,358 records (85.2%)
  0: 66,852 records (14.8%)


In [13]:
# =============================================================================
# 6. SAVE PROCESSED DATA / СОХРАНЕНИЕ ОБРАБОТАННЫХ ДАННЫХ
# =============================================================================

print("\n💾 Saving processed data...")

# Save to Excel
output_path = "processed_toxic_comments_ds.xlsx"
df.to_excel(output_path, index=False)
print(f"✅ Data saved to: {output_path}")
print(f"📊 Records saved: {len(df):,}")


💾 Saving processed data...
✅ Data saved to: processed_toxic_comments_ds.xlsx
📊 Records saved: 450,210
